In [123]:
"""
generate train data, validation data, and test data
"""
import os
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
import os
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
import helper
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import collections
import time

# Create Train and Target Data

In [7]:
# get codes
gallery_features_batch1 = np.load(open(r'gait_data/fc8_90/fc8_90_gallery_codes', mode='rb'))
probe_features_batch1 = np.load(open(r'gait_data/fc8_90/fc8_90_probe_codes', mode='rb'))
# feature shape
print("gallery length: {}".format(gallery_features_batch1.shape))
print("gallery shape: {}".format(gallery_features_batch1[0].shape))
print("")
print("probe length: {}".format(probe_features_batch1.shape))
print("probe shape: {}".format(probe_features_batch1[0].shape))

gallery length: (93,)
gallery shape: (32, 1000)

probe length: (93,)
probe shape: (31, 1000)


### Add Zero

In [37]:
gallery_features_batch1_add0 = np.zeros([len(gallery_features_batch1), 43, 1000], dtype=float)
probe_features_batch1_add0 = np.zeros([len(probe_features_batch1), 43, 1000], dtype=float)

g_seq_length = []
p_seq_length = []
for ii, (gallery, probe) in enumerate(zip(gallery_features_batch1, probe_features_batch1)):
    gallery_length = gallery.shape[0]
    probe_length = probe.shape[0]
    g_seq_length.append(gallery_length)
    p_seq_length.append(probe_length)
    
    gallery_features_batch1_add0[ii, :gallery_length, :] = gallery
    probe_features_batch1_add0[ii, :probe_length, :] = probe

### Create Target 1 Data

In [10]:
batch_1_target_1 = np.concatenate((gallery_features_batch1_add0, probe_features_batch1_add0), axis=1)
target_1 = np.ones([batch_1_target_1.shape[0]])

In [11]:
print("batch_1_target_1:", batch_1_target_1.shape)
print("")
print("target_1:", target_1.shape)

batch_1_target_1: (93, 86, 1000)

target_1: (93,)


### Create Target 0 Data
We create a same length batch with shuffled data
直到检测shuffle的idx没有重叠，函数会返回ok，否则打印error

In [20]:
idx = np.arange(len(gallery_features_batch1_add0))
np.random.shuffle(idx)
idx

array([28, 90, 79, 44, 41, 16, 38, 52, 31, 36, 78, 51, 73, 49, 25,  7, 12,
       57, 62, 17, 80, 55, 48, 74, 14, 89,  6, 40, 42, 37,  5,  0, 81,  9,
        2, 75, 43, 58, 54,  4, 32, 77, 34, 56, 82, 88, 76,  8, 61, 53, 23,
       72, 15, 35, 70, 68, 50, 29, 60, 67, 20, 30, 66, 11, 69, 71, 84, 85,
       18, 64, 47, 27, 92, 39, 87, 19, 10, 46, 45, 22, 33, 59,  1, 86, 83,
       13,  3, 24, 65, 91, 63, 21, 26])

In [21]:
def exam_shuffle_idx(idx):
    x =None
    for ii, index in enumerate(idx):
        if ii == index:
            x ='error'
            print(x)
            break
    if x != 'error':
        print("ok")
    return None
exam_shuffle_idx(idx)

ok


In [22]:
# create shuffled half_probe_features_batch1
shuffled_probe_features_batch1 = np.zeros(probe_features_batch1_add0.shape)
print(shuffled_probe_features_batch1.shape)
for ii, index in enumerate(idx):
    shuffled_probe_features_batch1[ii, :, :] = probe_features_batch1_add0[index]

(93, 43, 1000)


In [23]:
batch_1_target_0 = np.concatenate((gallery_features_batch1_add0, shuffled_probe_features_batch1), axis=1)
target_0 = np.zeros([batch_1_target_0.shape[0]])

In [24]:
batch_1_target_0[0][43:]

array([[ 0.14513677,  0.36747506,  0.29001927, ...,  0.12957594,
         0.35101527,  0.3333481 ],
       [ 0.11314621,  0.3523095 ,  0.27453646, ...,  0.09355041,
         0.36715832,  0.32119063],
       [ 0.08449331,  0.32923666,  0.31943917, ...,  0.13022023,
         0.36273172,  0.36481997],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
batch_1_target_1[28][43:]

array([[ 0.14513677,  0.36747506,  0.29001927, ...,  0.12957594,
         0.35101527,  0.3333481 ],
       [ 0.11314621,  0.3523095 ,  0.27453646, ...,  0.09355041,
         0.36715832,  0.32119063],
       [ 0.08449331,  0.32923666,  0.31943917, ...,  0.13022023,
         0.36273172,  0.36481997],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
# Concatenate half_batch_1_target_0 and half_batch_1_target_1 together
new_batch_1 = np.concatenate((batch_1_target_1, batch_1_target_0), axis=0)
new_batch_1.shape

(186, 86, 1000)

In [27]:
with open('gait_data/fc8_90/ordered_new_batch_1', 'wb') as f:
    np.save(f, new_batch_1)

In [29]:
target_1_and_0 = np.concatenate((target_1, target_0))

In [30]:
# Shuffle new_half_batch_1 to random order
idx2 = np.arange(len(new_batch_1))
np.random.shuffle(idx2)
idx2

array([ 78,  31,  59,  42,  68,  28, 180,  93, 162,  24,  49, 135,  47,
         7, 182,  40, 167,  43, 116, 169, 104,   6, 181,  62,  84, 145,
       138, 161, 185,  16, 105, 140, 121,  45, 110, 141, 125, 164, 152,
       132, 143,  71, 174,  46,   5, 112,  32, 156, 124,  76,  60, 160,
       113,  27, 147,  91, 111,  38,  29,  26, 114, 102,  70,  33,  12,
        10,  54, 165,  57,  25, 131,  92, 158, 144,   0,  82, 126,  17,
       146, 119, 148,  58,   4, 100, 133,  19, 170, 150, 115,  21,  67,
        94,  99,  48, 177, 101, 108,  98,  73,  83,  95,  20,  13, 106,
        34, 153,  14, 123,  18, 171,  81,  36,  22,  86, 134, 139, 184,
       151,   3, 129, 179,  97, 127, 107,   2, 157,  37, 172,  23, 178,
        96,  69, 149,  39,  77, 154, 103,  90,  44,  55,  65,  53,  11,
         9,   8,  56,  85,  35,  51, 166,  41,  52,  75,  89, 175, 173,
       142,   1, 176,  30,  79,  64, 168,  61, 120, 128, 130,  63,  80,
        50, 163,  15,  74, 117, 155, 183, 109,  66, 118,  88,  7

In [31]:
shuffled_new_batch_1 = np.ndarray(new_batch_1.shape, dtype=float)
shuffled_target_1_and_0 = np.ndarray(target_1_and_0.shape, dtype=float)
for ii, index in enumerate(idx2):
    shuffled_new_batch_1[ii, :, :] = new_batch_1[index]
    shuffled_target_1_and_0[ii] = target_1_and_0[index]

In [32]:
with open('gait_data/fc8_90/shuffled_new_batch_1', 'wb') as f:
    np.save(f, shuffled_new_batch_1)

In [33]:
one_hot_shuffled_new_batch_1_target = np.ndarray([186, 2], dtype=float)
for ii, each in enumerate(shuffled_target_1_and_0):
    if int(each) == 1:
        one_hot_shuffled_new_batch_1_target[ii, :] = 1, 0
    elif int(each) == 0:
        one_hot_shuffled_new_batch_1_target[ii, :] = 0, 1

In [34]:
with open('gait_data/fc8_90/one_hot_shuffled_new_batch_1_target', 'wb') as f:
    np.save(f, one_hot_shuffled_new_batch_1_target)

## Create seq_length

In [38]:
p_seq_length = np.array(p_seq_length)
shuffled_p_seq_length = np.zeros(p_seq_length.shape, dtype=int)
for ii, index in enumerate(idx):
    shuffled_p_seq_length[ii] = p_seq_length[index]

In [39]:
seq_column1 = np.reshape(np.concatenate([g_seq_length, g_seq_length]), [186, 1])
seq_column2 = np.reshape(np.concatenate([g_seq_length, shuffled_p_seq_length]), [186, 1])
seq_length = np.concatenate([seq_column1, seq_column2], axis=1)

shuffled_seq_length = np.zeros(seq_length.shape, dtype=int)
for ii, index in enumerate(idx2):
    shuffled_seq_length[ii] = seq_length[index]

In [40]:
with open('gait_data/fc8_90/shuffled_seq_length', 'wb') as f:
    np.save(f, shuffled_seq_length)

# Checkpoints 1

In [110]:
seq_length = np.load(open(r'gait_data/fc8_90/shuffled_seq_length', mode='rb'))
shuffled_new_batch_1 = np.load(open(r'gait_data/fc8_90/shuffled_new_batch_1', mode='rb'))
one_hot_shuffled_new_batch_1_target = np.load(open(r'gait_data/fc8_90/one_hot_shuffled_new_batch_1_target', mode='rb'))
print(shuffled_new_batch_1.shape)

(186, 86, 1000)


# Get Train and Validation data

In [111]:
print("data length: {}".format(len(shuffled_new_batch_1)))
print("every length has : {}".format(shuffled_new_batch_1.shape))

data length: 186
every length has : (186, 86, 1000)


In [112]:
split_frac = 0.8
split_num = int(split_frac*len(shuffled_new_batch_1))

train_x_raw, val_x_raw = shuffled_new_batch_1[:split_num], shuffled_new_batch_1[split_num:]
train_y_raw, val_y_raw = one_hot_shuffled_new_batch_1_target[:split_num], one_hot_shuffled_new_batch_1_target[split_num:]
train_seq_length, val_seq_length = seq_length[:split_num], seq_length[split_num:]

# print size
print("Train_x: {}".format(np.array(train_x_raw).shape))
print("Validation_x: {}".format(np.array(val_x_raw).shape))
print("Train_y: {}".format(np.array(train_y_raw).shape))
print("Validation_y: {}".format(np.array(val_y_raw).shape))
print("train_seq_length: {}".format(np.array(train_seq_length).shape))
print("val_seq_length: {}".format(np.array(val_seq_length).shape))

Train_x: (148, 86, 1000)
Validation_x: (38, 86, 1000)
Train_y: (148, 2)
Validation_y: (38, 2)
train_seq_length: (148, 2)
val_seq_length: (38, 2)


# According_len_get_outputs

In [113]:
def According_len_get_outputs(length_batch_1, length_batch_2, outputs_1, outputs_2):
    """
    length_batch_1 and length_batch_2 should minus one
    """
    new_outputs_1 = np.ndarray([outputs_1.shape[0], outputs_1.shape[2]], dtype=float)
    new_outputs_2 = np.ndarray([outputs_2.shape[0], outputs_2.shape[2]], dtype=float)
    
    for ii, (L1, L2, O1, O2) in enumerate(zip(length_batch_1, length_batch_2, outputs_1, outputs_2)):
        new_outputs_1[ii, :] = O1[L1, :]
        new_outputs_2[ii, :] = O2[L2, :]
        
    return new_outputs_1, new_outputs_2

In [124]:
a = np.array([[1, 2, 1],
              [2, 3, 2]])
b = np.array([[1, 1, 1],
              [4, 1, 1]])


10

In [137]:
def calculate_eulidean_distance(outputs_1, outputs_2):
    distance = np.ndarray([outputs_1.shape[0], 1], dtype=float)
    
    for ii, (a, b) in enumerate(zip(outputs_1, outputs_2)):
        distance[ii, :] = np.sum(np.square(a-b))
    return distance

# Build the graph

In [138]:
def build_rnn(batch_size=20, lstm_size=1000, num_layers=1, learning_rate=0.001, grad_clip=1):
    
    # reset graph
    tf.reset_default_graph()
    
    with tf.name_scope('inputs_1'):
        inputs_1 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_1")
    with tf.name_scope('inputs_2'):
        inputs_2 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_2")
    with tf.name_scope('targets'):
        target_ = tf.placeholder(tf.float32, [None, 2], name="targets")
        
    seq_1 = tf.placeholder(tf.float32, (None,), name="seq_1")
    seq_2 = tf.placeholder(tf.float32, (None,), name="seq_2")
    
    new_outputs_1 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_1")
    new_outputs_2 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_2")
    
    distance = tf.placeholder(tf.float32, [None, 1], name='distance')
    
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
#     seq_length = tf.placeholder(tf.int32, (None, ), name='seq_length')
    with tf.variable_scope("LSTM"):
#         with tf.name_scope("RNN_cells_1"):
            # Your basic LSTM cell
        #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
        #         lstm = tf.nn.relu(lstm)
        lstm_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_1 = tf.contrib.rnn.DropoutWrapper(lstm_1, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_1 = tf.contrib.rnn.MultiRNNCell([drop_1] * num_layers)

#         with tf.name_scope("RNN_init_state_1"):
            # Getting an initial state of all zeros
        initial_state_1 = cell_1.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_1"):
        outputs_1, final_state_1 = tf.nn.dynamic_rnn(cell_1, inputs_1, initial_state=initial_state_1, 
                                                     sequence_length=seq_1)

    with tf.variable_scope("LSTM", reuse=True):
#         with tf.name_scope("RNN_cells_2"):
            # Your basic LSTM cell
    #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
    #         lstm = tf.nn.relu(lstm)
        lstm_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_2 = tf.contrib.rnn.DropoutWrapper(lstm_2, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_2 = tf.contrib.rnn.MultiRNNCell([drop_2] * num_layers)

#         with tf.name_scope("RNN_init_state_2"):
            # Getting an initial state of all zeros
        initial_state_2 = cell_2.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_2"):
        outputs_2, final_state_2 = tf.nn.dynamic_rnn(cell_2, inputs_2, initial_state=initial_state_2, 
                                                     sequence_length=seq_2)

    with tf.name_scope('predictions'):
        # only choose final one
#         final_output_concat = tf.concat([outputs_1[:, -1], outputs_2[:, -1]], 1)

        # through the According_len_get_outputs
        # Eulidean distance
        logits = tf.contrib.layers.fully_connected(distance, 2)
        predictions = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', predictions)
        
    with tf.name_scope('cost'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_, logits=logits))
        tf.summary.scalar('cost', cost)
        
    with tf.name_scope('train'):
#         tvars = tf.trainable_variables()
#         grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip, name='clip')
#         train_op = tf.train.AdamOptimizer(learning_rate)
#         optimizer = train_op.apply_gradients(zip(grads, tvars))
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
    with tf.name_scope('saver'):
        saver = tf.train.Saver()
        
    merged = tf.summary.merge_all()
    
    export_nodes = ['inputs_1', 'inputs_2', 'target_', 'initial_state_1', 'initial_state_2', 'outputs_1', 'outputs_2',
                    'final_state_1', 'final_state_2', 'keep_prob', 'cost', 'logits', 'predictions', 'optimizer', 
                    'merged', 'saver', 'learning_rate', 'cell_1', 'cell_2', 'distance', 'seq_1', 'seq_2',
                    'new_outputs_1', 'new_outputs_2']
    
    Graph = collections.namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Get Batches

In [139]:
def get_batches(data, target, seq_length, batch_size):
    n_batches = len(data)//batch_size
    data = data[:batch_size * n_batches]
    target = target[:batch_size * n_batches]
    for ii in range(0, batch_size*n_batches, batch_size):
        data_batch = data[ii:ii + batch_size]
        target_batch = target[ii:ii + batch_size]
        seq_length_batch_1 = seq_length[ii:ii + batch_size, 0]
        seq_length_batch_2 = seq_length[ii:ii + batch_size, 1]
        
        yield data_batch, target_batch, seq_length_batch_1, seq_length_batch_2

# Accuracy

In [140]:
train_seq_length

array([[28, 28],
       [31, 31],
       [30, 30],
       [33, 33],
       [30, 30],
       [31, 31],
       [30, 32],
       [32, 31],
       [26, 33],
       [32, 32],
       [27, 27],
       [33, 34],
       [32, 32],
       [31, 31],
       [31, 34],
       [32, 32],
       [31, 31],
       [30, 30],
       [33, 30],
       [30, 32],
       [32, 32],
       [35, 35],
       [30, 30],
       [29, 29],
       [31, 31],
       [23, 33],
       [25, 31],
       [30, 32],
       [28, 32],
       [33, 33],
       [30, 32],
       [32, 30],
       [31, 35],
       [25, 25],
       [31, 29],
       [29, 30],
       [32, 31],
       [31, 32],
       [30, 31],
       [31, 33],
       [28, 33],
       [31, 31],
       [31, 27],
       [31, 31],
       [31, 31],
       [30, 31],
       [32, 32],
       [30, 33],
       [31, 31],
       [30, 30],
       [30, 30],
       [31, 30],
       [30, 28],
       [32, 32],
       [29, 32],
       [34, 34],
       [31, 30],
       [33, 33],
       [29, 29

# Train

In [141]:
train_x = train_x_raw
val_x = val_x_raw
train_y = train_y_raw
val_y = val_y_raw

In [142]:
train_x.shape

(148, 86, 1000)

In [143]:
def train(model, epochs, file_writer, save_string):

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = 1
        mean_val_Acc = 0
        mean_val_loss = 0
        count_Acc_not_increase_epochs = 0
        count_loss_not_decrease_epochs = 0
        Last_val_Acc = 0
        Last_val_loss = 0
        
        for e in range(epochs):
            for train_batch, target_batch, seq_length_batch_1, seq_length_batch_2 \
                             in get_batches(train_x, train_y, train_seq_length, batch_size):
                start = time.time()
#                 state_1 = sess.run(model.initial_state_1)
#                 state_2 = sess.run(model.initial_state_2)
#                 print(train_batch[:, :15, :].shape)
#                 print(target_batch)
    #             print('target: {}'.format(target_batch.shape))
                
                feed_1 = {
                        model.inputs_1: train_batch[:, :43, :],
                        model.inputs_2: train_batch[:, 43:, :],  
                        model.seq_1: seq_length_batch_1,
                        model.seq_2: seq_length_batch_2,
                        model.keep_prob: 1
                        }
            
#                 print(model.outputs_1.shape, model.outputs_1[:, -1], model.final_output_concat, 
#                       model.logits, model.predictions, model.cost)
                o_1, o_2 = sess.run([model.outputs_1, model.outputs_2], feed_dict=feed_1)
                
                real_outputs_1, real_outputs_2 = According_len_get_outputs(
                                               seq_length_batch_1 - 1, seq_length_batch_2 - 1, o_1, o_2)
            
                calculated_distance = calculate_eulidean_distance(real_outputs_1, real_outputs_2)
                
#                 print(o_1[:, 28], o_2[:, 28])
#                 print(real_outputs_1, real_outputs_2)
                feed_2 = {
                        model.target_: target_batch,
#                         model.keep_prob: 1,
#                         model.initial_state_1: state_1,
#                         model.initial_state_2: state_2,
                        model.learning_rate: lr,
#                         model.new_outputs_1: real_outputs_1,
#                         model.new_outputs_2: real_outputs_2
                        model.distance: calculated_distance
                        }
                
                loss, _, logi, pred, summary,  = sess.run(
                                [model.cost, model.optimizer, model.logits, model.predictions, model.merged]
                                , feed_dict=feed_2)
                print(pred)
#                 print(logi)
#                 print('pred: {}'.format(type(pred)))
    #             print('pred: {}'.format(np.count_nonzero(pred)))
    #             print('pred: {}'.format(np.sum(pred>0.1)))
    #             print('pred: {}'.format(pred.size))
    #             print('output: {}'.format(out[1, 39, :]))
                
                if iteration%5==0:
                    end = time.time()
#                     acc = calculate_accuracy(sess, pred, target_batch, sq_length)
                    print("Epoch: {}/{},".format(e+1, epochs),' ',
                          "Iteration: {},".format(iteration),' ',
                          "Train loss: {:.3f},".format(loss),' ',
                          "{:.1f}s /batch.".format((end-start)/5),' '
#                           "ACCuracy: %{:.3f}".format(acc)
                         )
                    
                    file_writer.add_summary(summary, iteration)
                
#                 if iteration%25==0:
#                     validation_loss = []
#                     validation_Acc = []
                    
#                     if batch_size >= len(val_x):
#                         val_batch_size = 30
#                     else: 
#                         val_batch_size = batch_size
                    
#                     for val_batch, val_target_batch in get_batches(val_x, val_y, val_batch_size):

#                         val_state_1 = sess.run(model.cell_1.zero_state(val_batch_size, tf.float32))
#                         val_state_2 = sess.run(model.cell_2.zero_state(val_batch_size, tf.float32))

#                         feed = {model.inputs_1: val_batch[:, :15, :],
#                                 model.inputs_2: val_batch[:, 15:, :],
#                                 model.target_: val_target_batch,
#                                 model.keep_prob: 1,
#                                 model.initial_state_1: val_state_1,
#                                 model.initial_state_2: val_state_2,
#                                }

#                         val_loss, val_pred = sess.run([model.cost, model.predictions], feed_dict=feed)
                        
# #                         val_acc = calculate_accuracy(sess, val_pred, val_target_batch, val_sq_length)

#                         validation_loss.append(val_loss)
                        
# #                         validation_Acc.append(val_acc)
                        
#                     Last_val_Acc = mean_val_Acc
#                     Last_val_loss = mean_val_loss
                        
#                     mean_val_loss = sum(validation_loss)/len(validation_loss)  
#                     mean_val_Acc = sum(validation_Acc)/len(validation_Acc)
#                     print()
#                     print("Validation loss: {:.3f},".format(mean_val_loss), ' ',
#                           "Validation Accuracy: %{:.3f}".format(mean_val_Acc))
#                     print()
                    
                iteration += 1
                
            # Early stopping  
#             if mean_val_Acc - Last_val_Acc <= -0.3:
#                 count_Acc_not_increase_epochs += 1
#             if Last_val_loss - mean_val_loss <= -0.01:
#                 count_loss_not_decrease_epochs += 1
            
#             if count_Acc_not_increase_epochs >= 5:
#                 break
#             if count_loss_not_decrease_epochs >= 5:
#                 break
                
        model.saver.save(sess, r"{}".format(save_string))

In [144]:
!mkdir checkpoints_identification

mkdir: checkpoints_identification: File exists


In [ ]:
epochs = 1000
batch_size = 100
for lstm_size in [3000]:
    for num_layers in [1]:
        for lr in [0.0005]:
                log_string = 'logs/4/lr={},nl={},ls={},bs={}'.format(lr, num_layers, lstm_size, batch_size)
                save_string = 'checkpoints\lr={}_nl={}_ls={}_bs={}.ckpt'.format(lr, num_layers, lstm_size, batch_size)
                
                writer = tf.summary.FileWriter(log_string)

                model = build_rnn(batch_size=batch_size, lstm_size=lstm_size, 
                                  num_layers=num_layers, learning_rate=lr, grad_clip=100)

                train(model, epochs, writer, save_string)
                
                print(' ')
                print(' ')
                print("leraning_rate={},num_layers={},lstm_size={},batch_size={} finished, saved".format(lr, num_layers, lstm_size, batch_size))
                print(' ')
                print(' ')

[[ 0.59136283  0.4086372 ]
 [ 0.79786444  0.2021355 ]
 [ 0.92679328  0.0732067 ]
 [ 0.53299987  0.46700019]
 [ 0.52313411  0.47686595]
 [ 0.52908456  0.47091541]
 [ 0.53580892  0.46419105]
 [ 0.53222674  0.46777323]
 [ 0.54456913  0.4554309 ]
 [ 0.52329391  0.47670603]
 [ 0.52706289  0.47293711]
 [ 0.53627473  0.4637253 ]
 [ 0.53188211  0.46811789]
 [ 0.52011949  0.47988045]
 [ 0.53037304  0.46962696]
 [ 0.53331196  0.46668798]
 [ 0.53685695  0.46314305]
 [ 0.80287796  0.19712204]
 [ 0.54209375  0.45790631]
 [ 0.5321455   0.46785447]
 [ 0.53365618  0.46634385]
 [ 0.52487475  0.47512522]
 [ 0.52968407  0.47031587]
 [ 0.52838194  0.47161806]
 [ 0.52820855  0.47179148]
 [ 0.58258229  0.41741768]
 [ 0.53980678  0.46019325]
 [ 0.53625506  0.463745  ]
 [ 0.52823573  0.47176427]
 [ 0.52395821  0.4760417 ]
 [ 0.55552834  0.44447166]
 [ 0.53278047  0.46721953]
 [ 0.56230259  0.43769741]
 [ 0.52132839  0.47867158]
 [ 0.53441691  0.46558312]
 [ 0.54526645  0.45473352]
 [ 0.52960157  0.47039843]
 